In [1]:
from init import *

In [57]:
start_date = '2018-01-01'
end_date = '2023-10-01'

indusrty_codes = []
for i in range(0,31):
    indusrty_codes.append(f"CI0050{str(i).rjust(2,'0')}.INDX")

indusrty_codes_convert = {'CI005001.INDX':'石油石化',
                            'CI005002.INDX':'煤炭',
                            'CI005003.INDX':'有色金属',
                            'CI005004.INDX':'电力及公用事业',
                            'CI005005.INDX':'钢铁',
                            'CI005006.INDX':'基础化工',
                            'CI005007.INDX':'建筑',
                            'CI005008.INDX':'建材',
                            'CI005009.INDX':'轻工制造',
                            'CI005010.INDX':'机械',
                            'CI005011.INDX':'电力设备及新能源',
                            'CI005012.INDX':'国防军工',
                            'CI005013.INDX':'汽车',
                            'CI005014.INDX':'商贸零售',
                            'CI005015.INDX':'消费者服务',
                            'CI005016.INDX':'家电',
                            'CI005017.INDX':'纺织服装',
                            'CI005018.INDX':'医药',
                            'CI005019.INDX':'食品饮料',
                            'CI005020.INDX':'农林牧渔',
                            'CI005021.INDX':'银行',
                            'CI005022.INDX':'非银行金融',
                            'CI005023.INDX':'房地产',
                            'CI005024.INDX':'交通运输',
                            'CI005025.INDX':'电子',
                            'CI005026.INDX':'通信',
                            'CI005027.INDX':'计算机',
                            'CI005028.INDX':'传媒',
                            'CI005029.INDX':'综合',
                            'CI005030.INDX':'综合金融'}

indusrty_names_convert = {}
for k,v in indusrty_codes_convert.items():
    indusrty_names_convert[v] = k
    
insts = all_instruments('CS')
# 获取股票池&研究周期
stock_list = sorted(insts[((insts['de_listed_date'] == '0000-00-00') | (insts['de_listed_date'] > start_date)) & (insts['listed_date'] <= end_date)].order_book_id.tolist())
date_list = pd.to_datetime(get_trading_dates(start_date,end_date))
# 构建行业分组
industry_map = get_industry_exposure(stock_list,date_list)

gen industry martix... 


In [3]:
def get_turnover_ratio(stock_list,start_date,end_date):
    def turnover_rate(order_book_ids,start_datea,end_date):
        return rqdatac.get_turnover_rate(order_book_ids,start_date,end_date,fields='today').today.unstack('order_book_id').reindex(columns=order_book_ids,index =pd.to_datetime(rqdatac.get_trading_dates(start_date,end_date)))

    DAILY_TURNOVER_RATE = UserDefinedLeafFactor('DAILY_TURNOVER_RATE',turnover_rate)
    turn_ratio = execute_factor(DAILY_TURNOVER_RATE- 1,stock_list,start_date,end_date)

    return turn_ratio

#### 一、动量

In [120]:
CLOSE = Factor('close')
mtm_5 = execute_factor(CLOSE/REF(CLOSE,5) - 1,stock_list,start_date,end_date)
# mtm_20 = execute_factor(CLOSE/REF(CLOSE,20) - 1,stock_list,start_date,end_date)
# 获取涨幅靠前标的的对应行业
mtm_5_top = mtm_5.mask(mtm_5.sub(mtm_5.quantile(0.9,axis = 1),axis = 0) < 0).stack().to_frame('mtm_5')
mtm_5_top['industry'] = industry_map
mtm_5_top.index.names = ['date','order_book_id']
# 整体市场数据分布
all_industry_count = industry_map.to_frame('industry').reset_index().groupby(['date','industry'])['order_book_id'].count()
# 计算占比
mtm_5_industry_count = mtm_5_top.reset_index().groupby(['date','industry']).order_book_id.count()
mtm_5_industry_ratio = (mtm_5_industry_count/all_industry_count).unstack('industry').fillna(0.0)

industry_close = get_price(indusrty_codes,start_date,end_date,fields='close').close.unstack('order_book_id')


def get_mtm(df,n):
    return df.pct_change(n)

def get_sharpe(df,n):
    df_pct = df.pct_change()
    return df_pct.rolling(n).mean()/df_pct.rolling(n).std()

def get_ir(df,n):
    benchmark_pct = get_price('000985.XSHG',
                          df.index[0],
                          df.index[-1],
                          fields='close').close.droplevel(0).pct_change()
    
    excess_df = df.sub(benchmark_pct,axis = 0)
    return excess_df.rolling(n).mean()/excess_df.rolling(n).std()

def get_distance(df,n):
    df_pct_abs = df.pct_change().abs()
    return df.pct_change(n)/df_pct_abs.rolling(n).sum()

def get_VaR(df,n):
    df_pct = df.pct_change() 
    #方差协方差法
    def VaR_VCM(ret,T = 10,X = 0.99):
        '''
        Parameters
        ----------
        ret : 资产日收益率
        T : 持有天数 
        X : 置信水平
        '''  
        mu = ret.mean()
        sig = ret.std()
        z = abs(st.norm.ppf(q=1-X))    
        return np.sqrt(T)*(z*sig-mu)
    
    return df_pct.rolling(n).apply(lambda x: VaR_VCM(x))

industry_mtm_60 = get_mtm(industry_close,60)
industry_sharpe_60 = get_sharpe(industry_close,60)
industry_ir_60 = get_ir(industry_close,60)
industry_distance_60 = get_distance(industry_close,60)
industry_VaR_60 = get_VaR(industry_close,60)

crowd_mtm_dict = {
                'mtm_5_industry_ratio':mtm_5_industry_ratio,
                'industry_mtm_60':industry_mtm_60,
                'industry_sharpe_60':industry_sharpe_60,
                'industry_ir_60':industry_ir_60,
                'industry_distance_60':industry_distance_60,
                'industry_VaR_60':industry_VaR_60
}

### 二、占比

In [9]:
# 成交额
total_turnover = get_price(stock_list,start_date,end_date,fields='total_turnover').swaplevel()
total_turnover['industry'] = industry_map
total_turnover_industry = total_turnover.reset_index().groupby(['date','industry'])['total_turnover'].sum().unstack('industry')
# 成交额占比
total_turnover_industry_ratio = total_turnover_industry.div(total_turnover_industry.sum(axis = 1),axis = 0)


# 市值
mc = execute_factor(Factor('market_cap_3'),stock_list,start_date,end_date).stack().to_frame('mc')
mc['industry'] = industry_map
mc_industry = mc.reset_index().groupby(['date','industry'])['mc'].sum().unstack('industry')
# 市值占比
mc_industry_ratio = mc_industry.div(mc_industry.sum(axis = 1),axis = 0)


# 换手率
turn_ratio = get_turnover_ratio(stock_list,start_date,end_date).stack().to_frame('turn_ratio')
turn_ratio['industry'] = industry_map
turn_ratio.index.names = ['date','order_book_id']
turn_ratio_industry = turn_ratio.reset_index().groupby(['date','industry'])['turn_ratio'].sum().unstack('industry')
# 换手率占比
turn_ratio_industry_ratio = turn_ratio_industry.div(turn_ratio_industry.sum(axis = 1),axis = 0)


crowd_ratio_dict = {
                    'total_turnover_industry':total_turnover_industry,
                    'mc_industry':mc_industry,
                    'turn_ratio_industry':turn_ratio_industry,
                    'total_turnover_industry_ratio':total_turnover_industry_ratio,
                    'mc_industry_ratio':mc_industry_ratio,
                    'turn_ratio_industry_ratio':turn_ratio_industry_ratio
}

#### 三、形态

In [16]:
# 行业内
CLOSE = Factor('close')
ret = execute_factor(CLOSE/REF(CLOSE,1) - 1,stock_list,start_date,end_date).stack()
ret.index.names = ['date','order_book_id']
benchmark = execute_factor(CLOSE/REF(CLOSE,1) - 1,['000985.XSHG'],start_date,end_date).stack().droplevel(1)
benchmark.index.names = ['date']
excess_return = (ret - benchmark).to_frame('excess_return')
excess_return['industry'] = industry_map

excess_return_skew = excess_return.reset_index().groupby(['date','industry'])['excess_return'].skew().unstack('industry')
excess_return_vol = excess_return.reset_index().groupby(['date','industry'])['excess_return'].std().unstack('industry')
excess_return_kurt = excess_return.reset_index().groupby(['date','industry'])['excess_return'].apply(lambda x: x.kurt()).unstack('industry')


# 行业_等权
CLOSE = Factor('close')
ret = execute_factor(CLOSE/REF(CLOSE,1) - 1,stock_list,start_date,end_date).stack().to_frame('ret')
ret.index.names = ['date','order_book_id']
ret['industry'] = industry_map
ret_industry = ret.reset_index().groupby(['date','industry'])['ret'].mean()
ret_industry.index.names = ['date','industry']

benchmark = execute_factor(CLOSE/REF(CLOSE,1) - 1,['000985.XSHG'],start_date,end_date).stack().droplevel(1)
benchmark.index.names = ['date']
excess_return_industry = (ret_industry - benchmark).unstack('industry')

excess_return_skew_indusrty = excess_return_industry.rolling(60).skew().dropna(how = 'all')
excess_return_vol_indusrty = excess_return_industry.rolling(60).std().dropna(how = 'all')
excess_return_kurt_indusrty = excess_return_industry.rolling(60).apply(lambda x: x.kurt()).dropna(how = 'all')

# 行业
industry_close = get_price(indusrty_codes,start_date,end_date,fields='close').close.unstack('order_book_id')
industry_pct = industry_close.pct_change()

def get_coskew(df,n):
    benchmark = get_price('000985.XSHG',
                          df.index[0],
                          df.index[-1],
                          fields='close').close.droplevel(0).pct_change()
    def get_resid(y):
        y = y.dropna()
        benchmark_temp = benchmark.loc[y.index[0]:y.index[-1]]
        resid_asset = sm.OLS(y,benchmark_temp,hasconst=False, missing='drop').fit().resid
        resid_market = benchmark_temp - benchmark_temp.mean()
        return ((resid_asset * resid_market) ** 2).mean() / np.sqrt((resid_asset ** 2).mean() * (resid_market ** 2).mean())
    
    df = df.rolling(n).apply(lambda x: get_resid(x))

    return df

industry_pct_skew_indusrty = industry_pct.rolling(60).skew().dropna(how = 'all')
industry_pct_coskew_indusrty = get_coskew(industry_pct,60).dropna(how = 'all')
industry_pct_kurt_indusrty = industry_pct.rolling(60).apply(lambda x: x.kurt()).dropna(how = 'all')

crowd_fig_dict = {
                'excess_return_skew':excess_return_skew,
                'excess_return_vol':excess_return_vol,
                'excess_return_kurt':excess_return_kurt,
                'excess_return_skew_indusrty':excess_return_skew_indusrty,
                'excess_return_vol_indusrty':excess_return_vol_indusrty,
                'excess_return_kurt_indusrty':excess_return_kurt_indusrty,
                'industry_pct_skew_indusrty':industry_pct_skew_indusrty,
                'industry_pct_coskew_indusrty':industry_pct_coskew_indusrty,
                'industry_pct_kurt_indusrty':industry_pct_kurt_indusrty
}

#### 四、相关性

In [20]:
def get_corr(x,y):
    x_to_y_corr = pd.DataFrame(index = date_list,columns = sorted(set(industry_map)))
    for i in range(60,len(y)):
        y_temp = y.iloc[i-60:i]
        total_turnover_industry_temp = x.iloc[i-60:i]
        x_to_y_corr.loc[y_temp.iloc[-1].name] = y_temp.corrwith(total_turnover_industry_temp)
    return x_to_y_corr


turn_ratio_to_mc_corr = get_corr(total_turnover_industry,mc_industry)
turn_ratio_to_mc_ratio_corr = get_corr(turn_ratio_industry_ratio,mc_industry_ratio)


crowd_corr_dict = {
                'turn_ratio_to_mc_corr':turn_ratio_to_mc_corr,
                'turn_ratio_to_mc_ratio_corr':turn_ratio_to_mc_ratio_corr,
}

#### 五、配对相关性

In [30]:
def get_pairwise(df):
    industry_names = sorted(set(industry_map))
    df_pair_corr = pd.DataFrame(index = date_list)
    for i in tqdm(industry_names):
        industry_df = df[df.industry == i]
        industry_df = industry_df.iloc[:,0].unstack('order_book_id')
        corr_sum = ((industry_df.rolling(60).corr().sum(axis = 1).replace(0,np.nan) - 1).unstack('order_book_id').sum(axis = 1))
        industry_pair_corr = 2/(industry_df.count(axis = 1) * (industry_df.count(axis = 1) - 1)) * corr_sum
        df_pair_corr[i] = industry_pair_corr
    return df_pair_corr


CLOSE = Factor('close')
ret = execute_factor(CLOSE/REF(CLOSE,1) - 1,stock_list,start_date,end_date).stack().to_frame('ret')
ret['industry'] = industry_map 
ret.index.names = ['date','order_book_id']

ret_pair_corr = get_pairwise(ret)

total_turnover = get_price(stock_list,start_date,end_date,fields='total_turnover').swaplevel()
total_turnover['industry'] = industry_map
total_turnover.index.names = ['date','order_book_id']
total_turnover_pair_corr = get_pairwise(total_turnover)


crowd_pairwise_dict = {
                'ret_pair_corr':ret_pair_corr,
                'total_turnover_pair_corr':total_turnover_pair_corr,
}

100%|██████████| 33/33 [06:07<00:00, 11.13s/it]


#### 六、乖离率

In [27]:
# 成交量
volume = get_price(stock_list,start_date,end_date,fields='volume').swaplevel()
volume['industry'] = industry_map
volume_industry = volume.reset_index().groupby(['date','industry'])['volume'].sum().unstack('industry')

# 成交额
total_turnover = get_price(stock_list,start_date,end_date,fields='total_turnover').swaplevel()
total_turnover['industry'] = industry_map
total_turnover_industry = total_turnover.reset_index().groupby(['date','industry'])['total_turnover'].sum().unstack('industry')

# 换手率
turn_ratio = get_turnover_ratio(stock_list,start_date,end_date).stack().to_frame('turn_ratio')
turn_ratio['industry'] = industry_map
turn_ratio.index.names = ['date','order_book_id']
turn_ratio_industry = turn_ratio.reset_index().groupby(['date','industry'])['turn_ratio'].sum().unstack('industry')

def get_bias(df,n):
    ma = df.rolling(n).mean()
    bais = df.sub(ma,axis = 0).div(ma,axis = 0) 
    return bais 

volume_industry_bais_60 = get_bias(volume_industry,60)
total_turnover_industry_bais_60 = get_bias(total_turnover_industry,60)
turn_ratio_industry_bais_60 = get_bias(turn_ratio_industry,60)


crowd_bais_dict = {
                'volume_industry_bais_60':volume_industry_bais_60,
                'total_turnover_industry_bais_60':total_turnover_industry_bais_60,
                'turn_ratio_industry_bais_60':turn_ratio_industry_bais_60
}

#### 七、波动率

In [29]:
industry_close = get_price(indusrty_codes,start_date,end_date,fields='close').close.unstack('order_book_id')
industry_pct = industry_close.pct_change()


industry_vol = industry_pct.rolling(60).std()
industry_vol_down = industry_pct.rolling(60).apply(lambda x: x[x < 0].std())

crowd_vol_dict = {
                'industry_vol':industry_vol,
                'industry_vol_down':industry_vol_down,
}

#### 汇总

In [121]:
crowd_dict = {**crowd_mtm_dict,
              **crowd_ratio_dict,
              **crowd_fig_dict,
              **crowd_corr_dict,
              **crowd_pairwise_dict,
              **crowd_bais_dict,
              **crowd_vol_dict
              }

# 列名统一
indusrty_name = list(indusrty_codes_convert.values())
for k,v in crowd_dict.items():
    if v.columns[0] in indusrty_name:
        df = v[indusrty_name]
        df.columns = [indusrty_names_convert[x] for x in df.columns.tolist()]
        crowd_dict[k] = df

pickle.dump(crowd_dict,open('crowd_dict.pkl','wb'))

In [35]:
def calc_p(df,n):
    def calculate_percentile(data, current_data):
        sorted_data = sorted(data)
        position = sorted_data.index(current_data) + 1

        return position / len(sorted_data)
    df = df.rolling(n).apply(lambda x: calculate_percentile(x,x.iloc[-1])).dropna(how = 'all')

    return df

def calc_z(df,n):
    df = ((df - df.rolling(n).mean())/(df.rolling(n).std())).dropna(how = 'all')

    return df

In [347]:
# 方向检验

industry_ret20 = get_price(indusrty_codes,
                           start_date,
                           end_date,
                           fields='close').close.unstack('order_book_id').pct_change(20).shift(-20).dropna(how = 'all')

param_record_z = pd.DataFrame()
param_record_p = pd.DataFrame()
for k,v in tqdm(crowd_dict.items()):
    df_z = calc_z(v,120)
    df_p = calc_p(v,120)
    inter_index = sorted(set(df_z.index) & set(industry_ret20.index))
    
    def get_param(df):
        recrod = pd.DataFrame(index = indusrty_codes,columns = ['param'])
        for i in indusrty_codes:
            y = industry_ret20.loc[inter_index,i].loc['2019-01-01':]
            x = df.loc[inter_index,i].loc['2019-01-01':]
            try:
                recrod.loc[i] = sm.OLS(y.astype(float),sm.add_constant(x.astype(float)),hasconst=False, missing='drop').fit().params[1]
            except:
                recrod.loc[i] = np.nan
        return recrod
    
    recrod_z = get_param(df_z)
    param_record_z = pd.concat([param_record_z,recrod_z.param.to_frame(k)],axis = 1)
    recrod_p = get_param(df_p)
    param_record_p = pd.concat([param_record_p,recrod_p.param.to_frame(k)],axis = 1)

param_record_z.mean().sort_values()

mc_industry                       -0.004528
ret_pair_corr                     -0.004215
total_turnover_industry_ratio      -0.00416
industry_vol                      -0.003763
turn_ratio_industry               -0.003485
turn_ratio_industry_ratio         -0.003409
excess_return_vol_indusrty        -0.003366
industry_VaR_60                   -0.003241
total_turnover_industry           -0.002838
industry_pct_coskew_indusrty      -0.002794
mc_industry_ratio                 -0.002428
industry_mtm_60                   -0.002327
industry_vol_down                  -0.00208
mtm_5_industry_ratio              -0.001874
turn_ratio_to_mc_ratio_corr       -0.001353
excess_return_kurt_indusrty       -0.001301
industry_distance_60              -0.001253
industry_sharpe_60                -0.001117
turn_ratio_to_mc_corr              -0.00085
volume_industry_bais_60           -0.000741
excess_return_skew_indusrty       -0.000568
industry_pct_kurt_indusrty        -0.000548
turn_ratio_industry_bais_60     

In [356]:
for k,v in crowd_dict.items():
    if param_record_z[k].mean() < 0:
        crowd_dict[k] = -1 * crowd_dict[k]

In [357]:
pickle.dump(crowd_dict,open('crowd_dict.pkl','wb'))